In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
from geopandas.tools import sjoin
import shapely
from shapely.geometry import Point
import pyreadr
import mercantile
from shapely.geometry import shape
import networkx as nx
import seaborn as sn
import pickle
import scipy.sparse as sp
from scipy.optimize import least_squares
import datetime
import lmfit
from sklearn import metrics
import math

# Import Data and Preprocessing

In [5]:
#################
# Load datasets #
#################

root_data = '/Users/ignaciosacristanbarba/Documents/M4R/Data'
root_results = '/Users/ignaciosacristanbarba/Documents/M4R/Results'

root_base = root_results+'/Base Network'

# Load LSCC
root_lscc = root_base+'/base_network_lscc.npz'
with open(root_lscc, 'rb') as handle:
        lscc_dict = pickle.load(handle)
        
##############################
# Generate DiGraph from data #
##############################

lscc = nx.from_dict_of_dicts(lscc_dict,create_using = nx.DiGraph)
lscc_nodes = list(lscc.nodes())
n_nodes = len(lscc_nodes)

# Store node numbers
node_numbers = {i : lscc_nodes[i] for i in range(n_nodes)}

# Load geom_dict
root_geom_dict = root_base+'/base_geom_dict.pickle'
with open(root_geom_dict, 'rb') as handle:
        geom_dict = pickle.load(handle)
# Gemo dict for node keys
geom_dict_numbers = {i : geom_dict[node_numbers[i]] for i in range(n_nodes)}

# Compute adjacency matrix and node list of LSCC
A_LSCC = nx.adjacency_matrix(lscc)
A_LSCC_array = A_LSCC.toarray()
lscc_nodes_list = np.asarray(list(lscc.nodes()))
s_lscc = len(lscc_nodes_list)

# Subtract self-loops
B_LSCC_array = A_LSCC - np.diag(np.diag(A_LSCC.toarray()))
B_LSCC = sp.csr_matrix(B_LSCC_array)

# Import NUTS shape files
root_data = '/Users/ignaciosacristanbarba/Documents/M4R/Data/'
root_map = root_data+'/NUTS_Level_3__January_2018__Boundaries-shp/NUTS_Level_3__January_2018__Boundaries.shp'
map_gdf = gpd.read_file(root_map)
map_gdf = map_gdf.to_crs("EPSG:3395")

# Get NUTS3 
cols = [0,2,3,4,5,6,7,8]
gdf_NUTS3 = map_gdf.drop(map_gdf.columns[cols],axis=1)
gdf_NUTS3.rename(columns={'nuts318cd': 'nuts'}, inplace=True)

# Get NUTS2
gdf_NUTS2 = gdf_NUTS3.copy()
gdf_NUTS2['nuts'] = [gdf_NUTS2['nuts'][i][:4] for i in range(179)]
gdf_NUTS2 = gdf_NUTS2.dissolve(by='nuts')

# Get NUTS1
gdf_NUTS1 = gdf_NUTS3.copy()
gdf_NUTS1['nuts'] = [gdf_NUTS1['nuts'][i][:3] for i in range(179)]
gdf_NUTS1 = gdf_NUTS1.dissolve(by='nuts')

# Load NUTS3 data
root_results = '/Users/ignaciosacristanbarba/Documents/M4R/Results/'
root_NUTS3_base = root_results+'/NUTS1_Base/'+'MOVEMENT_QUADKEY_NUTS3_GB.csv'
df_NUTS = pd.read_csv(root_NUTS3_base)
print('Shape of NUTS3 data:', df_NUTS.shape)

# Delete _ in front of quadkeys
quadkeys = [df_NUTS['quadkey'][i][1:] for i in range(df_NUTS.shape[0])]

# Get NUTS3 region names
NUTS3 = df_NUTS.columns.values.tolist()[1:]

# Get NUTS2 regions
to_NUTS2 = {NUTS3[i] : NUTS3[i][:4] for i in range(len(NUTS3))}
NUTS2_index = {list(to_NUTS2.values())[i] : i for i in range(len(to_NUTS2))}

# Get NUTS1 regions
to_NUTS1 = {NUTS3[i] : NUTS3[i][:3] for i in range(len(NUTS3))}
NUTS1_index = {list(to_NUTS1.values())[i] : i for i in range(len(to_NUTS1))}

# Compute for each quadkey NUTS3 region by max vote 
X = np.asarray(df_NUTS.iloc[:,1:])
max_rule = np.argmax(X,axis = 1)
print('The max is obtained in average with:',
      np.around(np.max(X,axis = 1).mean(),2) )

# Create dictionary from quadkeys to NUTS3 region id's
quadkey_NUTS3 = {quadkeys[i] : max_rule[i] for i in range(df_NUTS.shape[0])}

# Generate NUTS3 Id's for LSCC nodes
NUTS3_id = np.asarray([quadkey_NUTS3[list(lscc.nodes())[i]] for i in range(n_nodes)])
print('NUTS3 communities in LSCC:', len(set(NUTS3_id)))

# Generate NUTS2 Id's
NUTS2_id = np.asarray([ NUTS2_index[to_NUTS2[NUTS3[NUTS3_id[i]]]] for i in range(n_nodes)])
print('NUTS2 communities in LSCC:', len(set(NUTS2_id)))

# Generate NUTS1 Id's
NUTS1_id = np.asarray([ NUTS1_index[to_NUTS1[NUTS3[NUTS3_id[i]]]] for i in range(n_nodes)])
print('NUTS1 communities in LSCC:', len(set(NUTS1_id)))

Shape of NUTS3 data: (5436, 180)
The max is obtained in average with: 0.93
NUTS3 communities in LSCC: 170
NUTS2 communities in LSCC: 42
NUTS1 communities in LSCC: 12


In [3]:
###############
# Import data #
###############

root_data = '/Users/ignaciosacristanbarba/Documents/M4R/Data'
root_results = '/Users/ignaciosacristanbarba/Documents/M4R/Results'

df = pd.read_csv(root_results+'/Timeseries/FilteredDataFrame')
# Converting the quadkeys to strings
df['start_quadkey'] = df['start_quadkey'].astype(str)
df['end_quadkey'] = df['end_quadkey'].astype(str)
# adding a leading '0' to quadkeys beginning with 3 so it maps on to web mercator
df.loc[df['start_quadkey'].str[:1] == '3', 'start_quadkey'] = '0'+df['start_quadkey']
df.loc[df['end_quadkey'].str[:1] == '3', 'end_quadkey'] = '0'+df['end_quadkey']

# Replace nan by 0
df = df.fillna(0)

###########################
# Get dates of timeseries #
###########################

# Get start and end dates
start_date = df.columns.values[2][:10]
end_date = df.columns.values[-1][:10]

# Generate DatetimeIndex
days = pd.date_range(start=start_date, end=end_date).date
n_days = len(days)
days_dm = np.asarray([str(days[i])[5:] for i in range(n_days)])
days_week = np.asarray(pd.date_range(start=start_date, end=end_date).weekofyear, dtype='int')
weeks = np.arange(days_week.min(),days_week.max())

# Store lockdown-date
lockdown_date = pd.to_datetime('20200324', format='%Y%m%d', errors='ignore')
lockdown_date_number = np.argwhere(days == lockdown_date).flatten()[0]

# Store second lockdown-date
second_lockdown_date = pd.to_datetime('20201105', format='%Y%m%d', errors='ignore')
second_lockdown_date_number = np.argwhere(days == second_lockdown_date).flatten()[0]


# compute id's for NUTS1, 2 and 3 and insert them in the dataframe
df2 = df.copy()

NUTS3_id_start = np.asarray([quadkey_NUTS3[df2['start_quadkey'][i]] for i in range(df2.shape[0])])
NUTS2_id_start = np.asarray([ NUTS2_index[to_NUTS2[NUTS3[NUTS3_id_start[i]]]] for i in range(df2.shape[0])])
NUTS1_id_start = np.asarray([ NUTS1_index[to_NUTS1[NUTS3[NUTS3_id_start[i]]]] for i in range(df2.shape[0])])

NUTS3_id_end = np.asarray([quadkey_NUTS3[df2['end_quadkey'][i]] for i in range(df2.shape[0])])
NUTS2_id_end = np.asarray([ NUTS2_index[to_NUTS2[NUTS3[NUTS3_id_end[i]]]] for i in range(df2.shape[0])])
NUTS1_id_end = np.asarray([ NUTS1_index[to_NUTS1[NUTS3[NUTS3_id_end[i]]]] for i in range(df2.shape[0])])

df2.insert(0,'start_NUTS1',NUTS1_id_start)
df2.insert(1,'end_NUTS1',NUTS1_id_end)

#function to get keys by the values in a dictionary
def getKeysByValues(dictOfElements, listOfValues):
    key_list = list(dictOfElements.keys())
    val_list = list(dictOfElements.values())
    listOfKeys = []
    for item  in listOfValues:
        position = val_list.index(item)
        listOfKeys.append(key_list[position])
    return  listOfKeys 

# numbers of NUTS1 and create dictionary
NUTS1_no = df2.start_NUTS1.unique()
NUTS1_no_dict = dict(zip(NUTS1_no, range(len(NUTS1_no))))

start_NUTS1_abbr = getKeysByValues(NUTS1_index,df2['start_NUTS1'])
end_NUTS1_abbr = getKeysByValues(NUTS1_index,df2['end_NUTS1'])

df2['start_NUTS1'] = np.asarray([ NUTS1_no_dict[df2['start_NUTS1'][i]] for i in range(df2.shape[0])])
df2['end_NUTS1'] = np.asarray([ NUTS1_no_dict[df2['end_NUTS1'][i]] for i in range(df2.shape[0])])

# dictionary to name
NUTS1_abbr_to_name_dict = {
    "UKC": "North East",
    "UKD": "North West",
    "UKE": "Yorkshire",
    "UKF": "East Midlands",
    "UKG":"West Midlands",
    "UKH":"East of England",
    "UKI":"London",
    "UKJ":"South East",
    "UKK":"South West",
    "UKL":"Wales",
    "UKM":"Scotland",
    "UKN":"Northern Ireland",
}

# start and end names of NUTS1 regions
start_NUTS1_name = [NUTS1_abbr_to_name_dict[start_NUTS1_abbr[i]] for i in range(len(start_NUTS1_abbr))]
end_NUTS1_name = [NUTS1_abbr_to_name_dict[end_NUTS1_abbr[i]] for i in range(len(end_NUTS1_abbr))]

# create different dataframes for each NUTS1
dfs_NUTS1 = []
NUTS1_regions_indices = []
for i in range(len(NUTS1_no_dict)):
    #both are start and end are inside that region
    df_tmp = df2.loc[(df2['start_NUTS1'] == i) & (df2['end_NUTS1'] == i)]
    dfs_NUTS1.append(df_tmp)
    NUTS1_regions_indices.append(df_tmp.index)

NUTS1_numbers = getKeysByValues(NUTS1_no_dict,list(NUTS1_no_dict.values()))
NUTS1_abbr = getKeysByValues(NUTS1_index,NUTS1_numbers)
NUTS1_names = [NUTS1_abbr_to_name_dict[NUTS1_abbr[i]] for i in range(len(NUTS1_abbr))]

# For each dataframe (region), compute lscc base

In [52]:
root_results = '/Users/ignaciosacristanbarba/Documents/M4R/Results'
root_base = root_results+'/NUTS1_Base/'

NUTS1_networks_lscc = []
NUTS1_D_geom = []
NUTS1_geom_dict = []

for i,df in enumerate(dfs_NUTS1):
    df2 = df.copy()
    # Converting the quadkeys to strings
    df2['start_quadkey'] = df['start_quadkey'].astype(str)
    df2['end_quadkey'] = df['end_quadkey'].astype(str)
    # adding a leading '0' to quadkeys beginning with 3 so it maps on to web mercator
    df2.loc[df2['start_quadkey'].str[:1] == '3', 'start_quadkey'] = '0'+df['start_quadkey']
    df2.loc[df2['end_quadkey'].str[:1] == '3', 'end_quadkey'] = '0'+df['end_quadkey']

    df = df2
    # Replace nan by 0
    df = df.fillna(0)


    ##################
    # Aggregate data #
    ##################

    df = df.reset_index()
       
    # Get the quadkey columns
    df_key = df.iloc[:,3:5]

    M = df.shape[1]-3
    days = []

    for i in range(1,M,3):

        df_mov = df.iloc[:,i+1:i+4]
        df_sum = pd.Series(df_mov.sum(axis=1),name='movement')
        days.append(df_sum)

    # Compute weekly average
    all_days = days[0]

    for i in range(1,len(days)):
        all_days += days[i]

    all_days = all_days/len(days)
    df_mean = pd.concat([df_key,all_days],axis=1)

    ####################
    # Generate DiGraph #
    ####################

    N = len(df_mean)
    G = nx.DiGraph()

    # for each row, add nodes and weighted edge
    for i in range(0,N):
        start = df_mean['start_quadkey'][i]
        end = df_mean['end_quadkey'][i]
        weight = df_mean['movement'][i]
        G.add_node(start)
        G.add_node(end)
        if weight > 0.0:
            G.add_weighted_edges_from([(start, end, weight)] )

    ###############
    # Store Graph #
    ###############
    G_dict = nx.to_dict_of_dicts(G)
    
    # Create subgraph of LSCC and save adjacency matrix
    lscc_nodes = max(nx.strongly_connected_components(G), key=len)
    lscc = G.subgraph(lscc_nodes)
    lscc_dict = nx.to_dict_of_dicts(lscc)
    
    NUTS1_networks_lscc.append(lscc_dict)
    
    
    ###################################
    
    ####################################
    # Compute geometric node positions #
    ####################################

    # Get polygons from quadkeys
    quadkeys = list(G.nodes)
    n_nodes = len(quadkeys)
    polys = []

    #Iterates over the quadkeys to extract the tiles
    for quadkey in quadkeys:
        tile = mercantile.feature(mercantile.quadkey_to_tile(quadkey), projected = 'web mercator')
        polys.append(tile.get('geometry'))
    geom = [shape(i) for i in polys]

    #  add polygon centroids as node attributes 
    geom_dict = {quadkeys[i] : list(geom[i].centroid.bounds[:2]) for i in range(0,n_nodes)}
    nx.set_node_attributes(G,geom_dict,'geom')

    # Save geom_dict to file
    NUTS1_geom_dict.append(geom_dict)
    # Store node numbers
    node_numbers = {i : list(G.nodes())[i] for i in range(n_nodes)}
    node_dict = {list(G.nodes)[i] : i for i in range(n_nodes)}

    # Gemo dict for node keys
    geom_dict_numbers = {i : geom_dict[node_numbers[i]] for i in range(n_nodes)}

    # Store node attributes in data frame
    node_attributes = pd.DataFrame({'key' : quadkeys, 'geom' : geom_dict.values()})

    ###################################
    # Compute distances between tiles #
    ###################################

    gdf = gpd.GeoDataFrame({'geometry':geom, 'quadkey':quadkeys}, crs = "EPSG:4326")

    def getXY(pt):
        return (pt.x, pt.y)
    centroidseries = gdf['geometry'].centroid
    gdf['LONGITUDE'],gdf['LATITUDE'] = [list(t) for t in zip(*map(getXY, centroidseries))]

    from math import radians, cos, sin, asin, sqrt

    pt_lat = dict(zip(gdf['quadkey'], gdf['LATITUDE']))
    pt_long = dict(zip(gdf['quadkey'], gdf['LONGITUDE']))

    pt_lat_r = {k: math.radians(v) for k, v in pt_lat.items()}
    pt_long_r = {k: math.radians(v) for k, v in pt_long.items()}

    pt_tup_1 = []
    pt_tup_2 = []
    dist_tup = []

    # Compute distance in km with Haversine formula
    for i in pt_lat_r.keys():
        for j in pt_lat_r.keys():
            pt_tup_1.append(i)
            pt_tup_2.append(j)        
            dist_tup.append(6367 * (2 * asin(sqrt(sin((pt_lat_r.get(i) - pt_lat_r.get(j))/2)**2 + cos((pt_lat_r.get(i))) * cos((pt_lat_r.get(j))) * sin((pt_long_r.get(i) - pt_long_r.get(j))/2)**2))))

    #Creates Dataframe from tuple
    Dist = pd.DataFrame({'start_quadkey': pt_tup_1,'end_quadkey': pt_tup_2,'DISTANCE': dist_tup})

    # Compute distance matrix D_geom
    D_geom = np.zeros((n_nodes,n_nodes))
    for i in range(n_nodes):
        D_geom[i,:] = Dist['DISTANCE'][i*n_nodes:(i+1)*n_nodes]
        
    NUTS1_D_geom.append(D_geom)


with open(root_base+'NUTS1_base_geom_dict.pickle', 'wb') as handle:
    pickle.dump(NUTS1_geom_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(root_base+'/NUTS1_base_D_geom.pickle', 'wb') as handle:
    pickle.dump(NUTS1_D_geom, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(root_base+'/NUTS1_base_networks_lscc.npz', 'wb') as handle:
    pickle.dump(NUTS1_networks_lscc, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:120: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [63]:
len(NUTS1_networks_lscc[11])

44